# 좌표하강 알고리즘 (Coordinate Descent Algorithm)
잔차제곱합에 기반한 목적함수를 다음과 같이 정의하자.
$$
R(\beta) = \sum_{i=1}^n \left(y_i - \sum_{j=1}^p \beta_j x_{ij} \right)^2
$$
해당 목적함수를 최소화하기 위해 최소제곱법을 활용할 수도 있지만 좌표하강 알고리즘을 활용할 수도 있다. 
좌표하강 알고리즘은 하나의 계수를 제외한 나머지 계수들을 고정한 채, 하나씩 차례로 갱신하는 알고리즘으로 이해할 수 있다.
즉, 좌표하강 알고리즘은 일차원 최소화 문제를 차례로 해결하는 것이며, 다음 식을 통해 계수들이 갱신된다.
\begin{equation}
\tilde{\beta}_k \leftarrow \text{argmin}_{\beta_k} \sum_{i=1}^n \left(y_i - \sum_{j \neq k}^p \tilde{\beta}_j x_{ij} - \beta_k x_{ik} \right)^2,
\quad k = 1, \cdots, p
\end{equation}
좌표하강 알고리즘은 일차원 문제를 차례로 해결하므로, 상대적으로 문제를 해결하기 쉽고 역행렬을 구하지 않아도 된다. 
또한, $\ell_1$-길이 유형의 정규화 방법에 기반한 문제를 해결하는 경우, 해당 알고리즘의 유용성이 극대화된다.

## 일차원 최소화 문제
먼저 공식을 구체화하면 된다. 깔끔한 공식 유도를 위해 
$$
r_{ik} = y_i - \sum_{j \neq k}^p \tilde{\beta}_j x_{ij} \quad \text{and} \quad  
z_{ik} = x_{ik}
$$
라 하자. 
그러면 일차원 최소화 문제는 다음과 같다.
$$
\text{argmin}_{\beta_k} \sum_{i=1}^n \left(r_{ik} - \beta_k z_{ik} \right)^2, \quad k = 1, \cdots, p
$$
이 문제는 일차원 quadratic form 의 최적화 문제를 해결하는 것과 동일하고, 각 계수를 
$$
\tilde{\beta}_k \leftarrow \frac{\sum_{i=1}^n r_{ik} z_{ik}}{\sum_{i=1}^n z_{ik}^2}, \quad k = 1, \cdots, p
$$
로 갱신한다.

# 코드
코드를 작성할 때, 모듈을 세분화하여 각 모듈이 잘 작동하는지 확인하는 것이 중요하다고 생각한다.
이 작업을 수행하지 않으면 디버깅에 어려움을 겪을 수 있으며, 코드를 처음부터 다시 작성해야할 수도 있다.
코드를 작성하기 전, 세분화를 어떻게 할 것인지 설계를 해보는 것도 좋을 것이라 생각한다.
여기서는 예제로 갱신 공식을 모듈로 가정하고 테스트하는 작업을 간단하게 진행하였고 바로 좌표하강 알고리즘 코드를 작성하였다.
### 1. 갱신 공식

In [95]:
import numpy as np
def solution_cda(z, r):
    if not all(z):
        return 0
    else:
        return sum(r * z) / sum(z * z)

### 테스트
코드를 작성한 후에 간단한 setup 을 설정하고 코드가 맞게 작성되었는지 확인한다.
다음을 가정하자. 
$$
r = (1, 2, 3) \quad \text{and} \quad z = (1, 1, 1)
$$
그러면 
$$
\sum_{i=1}^n r_ik z_ik = 1 + 2 + 3 = 6
\quad
\text{and}
\quad
\sum_{i=1}^n z_ik^2 = 1 + 1 + 1 = 3 
$$
이고 solution_cda 함수의 반환값은 
$$
\frac{6}{3} = 2 
$$
가 되어야 한다. 

In [96]:
r = np.array([1, 2, 3])
z = np.array([1, 1, 1])
print(solution_cda(z, r))

2.0


## 2. 좌표하강 알고리즘
모든 계수들이 한 차례 갱신되는 경우, 한 번의 반복으로 간주한다. 
반복을 진행하며, 최소화해야하는 목적함수의 변화량이 적은 경우 반복을 멈추는 좌표하강 알고리즘 코드를 작성하였다.

In [112]:
def cda(x, y, n_iter = 1000, thres = 1e-08):
    n, p = x.shape
    beta = np.zeros(p)
    fitted_values = np.dot(x, beta)
    residuals = y - fitted_values
    rss_before = sum(residuals * residuals)
    for iter in range(n_iter):
        print(iter, "th iteration runs")
        for j in range(p):
            z = x[:, j]
            r = residuals + beta[j] * z
            beta[j] = solution_cda(z, r)
            residuals = r - beta[j] * z
        rss = sum(residuals * residuals)
        print("rss =", round(rss, 5), "\n")
        if rss_before - rss < thres:
            break
        else:
            rss_before = rss
    return beta

### 테스트
간단하게 회귀 문제를 통해 테스트를 진행했다. 
다음 회귀모형으로부터 데이터를 생성하였다.
$$
y_i = 2 x_{i1} + 3 x_{i2} + \varepsilon_i, \quad i = 1, \ldots, 200
$$
여기서 좌표하강 알고리즘을 통해 계산한 계수 추정치가 (2, 3) 에 가까워지는지 확인해보았다.
$20$ 번의 반복만에 (2, 3) 으로 수렴한 것을 다음 코드로부터 확인할 수 있다. 

In [118]:
# 데이터 생성
np.random.seed(1)
n = 200
p = 2
x = np.zeros(n * p).reshape(n, p)
for j in range(p):
    x[:, j] = np.random.uniform(0, 1, n)
f = 2 * x[:, 0] + 3 * x[:, 1]
y = f + np.random.normal(0, 0.01, n)

In [119]:
print(cda(x, y))

0 th iteration runs
rss = 161.73395 

1 th iteration runs
rss = 46.54467 

2 th iteration runs
rss = 13.40528 

3 th iteration runs
rss = 3.87124 

4 th iteration runs
rss = 1.12834 

5 th iteration runs
rss = 0.33922 

6 th iteration runs
rss = 0.1122 

7 th iteration runs
rss = 0.04688 

8 th iteration runs
rss = 0.02809 

9 th iteration runs
rss = 0.02269 

10 th iteration runs
rss = 0.02113 

11 th iteration runs
rss = 0.02068 

12 th iteration runs
rss = 0.02055 

13 th iteration runs
rss = 0.02052 

14 th iteration runs
rss = 0.02051 

15 th iteration runs
rss = 0.0205 

16 th iteration runs
rss = 0.0205 

17 th iteration runs
rss = 0.0205 

18 th iteration runs
rss = 0.0205 

19 th iteration runs
rss = 0.0205 

20 th iteration runs
rss = 0.0205 

[2.00143933 2.99872477]
